In [60]:
import requests
import pandas as pd
import yfinance as yf



def get_supported_xstocks():
    url = "https://api.coingecko.com/api/v3/coins/list"
    response = requests.get(url)
    data = response.json()

    stocks = [coin for coin in data if 'stock' in coin['id']]
    test_df = pd.DataFrame(stocks)
    return test_df


def get_stock_fundamentals(symbol):
    stock = yf.Ticker(symbol)
    info = stock.info

    data = {
        "symbol": symbol,
        "company_name": info.get("longName"),
        "current_price": info.get("currentPrice"),
        "market_cap": info.get("marketCap"),
        "pe_ratio": info.get("trailingPE"),
        "dividend_yield": info.get("dividendYield"),
        "next_earnings_date": info.get("earningsTimestamp"),
    }
    return data



In [61]:


symbols = ["TSLA", "AAPL", "GOOGL", "MSFT", "NVDA", "AMZN", "META", "NFLX"]

all_data = [get_stock_fundamentals(symbol) for symbol in symbols]

test_df = pd.DataFrame(all_data)
test_df

,symbol,company_name,current_price,market_cap,pe_ratio,dividend_yield,next_earnings_date
0,TSLA,"Tesla, Inc.",401.99,1336946065408,277.234470,NaN,1761163200
1,AAPL,Apple Inc.,272.95,4033205764096,36.637585,0.38,1761854400
2,GOOGL,Alphabet Inc.,278.57,3364478255104,27.472387,0.30,1761768000
3,MSFT,Microsoft Corporation,503.29,3740641525760,35.846867,0.72,1761768000
4,NVDA,NVIDIA Corporation,186.86,4549480808448,53.236470,0.02,1763586000
5,AMZN,"Amazon.com, Inc.",237.58,2539781619712,33.556500,NaN,1761854400
6,META,"Meta Platforms, Inc.",609.89,1537244659712,26.950510,0.34,1761768000
7,NFLX,"Netflix, Inc.",1154.23,489084551168,48.314358,NaN,1761076800


In [62]:

test_df.isna().sum()


symbol                0
company_name          0
current_price         0
market_cap            0
pe_ratio              0
dividend_yield        3
next_earnings_date    0
dtype: int64

In [63]:
test_df['dividend_yield']=test_df['dividend_yield'].fillna(0)
test_df.isna().sum()

symbol                0
company_name          0
current_price         0
market_cap            0
pe_ratio              0
dividend_yield        0
next_earnings_date    0
dtype: int64

In [64]:
test_df['current_price'] = test_df['current_price'].round(2)
test_df['market_cap'] = (test_df['market_cap'] / 1e9).round(2)  # Convert to Billions
test_df['pe_ratio'] = test_df['pe_ratio'].round(2)
test_df['dividend_yield'] = (test_df['dividend_yield'] * 100).round(2)  # Convert to %
test_df


,symbol,company_name,current_price,market_cap,pe_ratio,dividend_yield,next_earnings_date
0,TSLA,"Tesla, Inc.",401.99,1336.95,277.23,0.0,1761163200
1,AAPL,Apple Inc.,272.95,4033.21,36.64,38.0,1761854400
2,GOOGL,Alphabet Inc.,278.57,3364.48,27.47,30.0,1761768000
3,MSFT,Microsoft Corporation,503.29,3740.64,35.85,72.0,1761768000
4,NVDA,NVIDIA Corporation,186.86,4549.48,53.24,2.0,1763586000
5,AMZN,"Amazon.com, Inc.",237.58,2539.78,33.56,0.0,1761854400
6,META,"Meta Platforms, Inc.",609.89,1537.24,26.95,34.0,1761768000
7,NFLX,"Netflix, Inc.",1154.23,489.08,48.31,0.0,1761076800


In [65]:
test_df.rename(columns={
    'symbol': 'Symbol',
    'company_name': 'Company Name',
    'current_price': 'Price (USD)',
    'market_cap': 'Market Cap (Billion USD)',
    'pe_ratio': 'P/E Ratio',
    'dividend_yield': 'Dividend Yield (%)',
    'next_earnings_date': 'Next Earnings Date'
}, inplace=True)
test_df

,Symbol,Company Name,Price (USD),Market Cap (Billion USD),P/E Ratio,Dividend Yield (%),Next Earnings Date
0,TSLA,"Tesla, Inc.",401.99,1336.95,277.23,0.0,1761163200
1,AAPL,Apple Inc.,272.95,4033.21,36.64,38.0,1761854400
2,GOOGL,Alphabet Inc.,278.57,3364.48,27.47,30.0,1761768000
3,MSFT,Microsoft Corporation,503.29,3740.64,35.85,72.0,1761768000
4,NVDA,NVIDIA Corporation,186.86,4549.48,53.24,2.0,1763586000
5,AMZN,"Amazon.com, Inc.",237.58,2539.78,33.56,0.0,1761854400
6,META,"Meta Platforms, Inc.",609.89,1537.24,26.95,34.0,1761768000
7,NFLX,"Netflix, Inc.",1154.23,489.08,48.31,0.0,1761076800


In [66]:
def generate_market_insight(row):
    insights = []

    if row['P/E Ratio'] > 30:
        insights.append("appears overvalued based on high P/E ratio")
    elif row['P/E Ratio'] < 15:
        insights.append("appears undervalued based on low P/E ratio")
    else:
        insights.append("is fairly valued")

    if row['Dividend Yield (%)'] > 2:
        insights.append("offers a good dividend yield")
    elif row['Dividend Yield (%)'] == 0:
        insights.append("does not pay dividends")

    if row['Market Cap (Billion USD)'] > 500:
        insights.append("is a large-cap stock")
    else:
        insights.append("is a mid or small-cap company")

    return f"{row['Company Name']} {', '.join(insights)}."


In [67]:
test_df["Market Insight"] = test_df.apply(generate_market_insight, axis=1)
print(test_df[["Company Name", "Market Insight"]])



            Company Name                                     Market Insight
0            Tesla, Inc.  Tesla, Inc. appears overvalued based on high P...
1             Apple Inc.  Apple Inc. appears overvalued based on high P/...
2          Alphabet Inc.  Alphabet Inc. is fairly valued, offers a good ...
3  Microsoft Corporation  Microsoft Corporation appears overvalued based...
4     NVIDIA Corporation  NVIDIA Corporation appears overvalued based on...
5       Amazon.com, Inc.  Amazon.com, Inc. appears overvalued based on h...
6   Meta Platforms, Inc.  Meta Platforms, Inc. is fairly valued, offers ...
7          Netflix, Inc.  Netflix, Inc. appears overvalued based on high...


In [ ]:
import os
GROQ_API_KEY ="your_groq_api_key_here"  # Replace with your actual Groq API key

In [69]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq 
from tqdm import tqdm  # for progress bar
from langchain_core.messages import HumanMessage

# Initialize Groq model
llm = ChatGroq(model="llama-3.3-70b-versatile",temperature=0, api_key=GROQ_API_KEY)

def generate_ai_insight(row):
    prompt = f"""
    You are a financial analyst. Analyze the following stock data and provide
    a short, clear investment insight (2-3 sentences).

    Company: {row['Company Name']}
    Symbol: {row['Symbol']}
    Current Price: ${row['Price (USD)']}
    Market Cap: {row['Market Cap (Billion USD)']} Billion USD
    P/E Ratio: {row['P/E Ratio']}
    Dividend Yield: {row['Dividend Yield (%)']}%
    Next Earnings Date: {row['Next Earnings Date']}

    Focus on:
    - Whether it looks undervalued or overvalued
    - Growth potential or risk
    - Key investor takeaway
    """

    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()

test_df["AI Insight"] = test_df.apply(generate_ai_insight, axis=1)
test_df[["Company Name", "AI Insight"]]


,Company Name,AI Insight
0,"Tesla, Inc.","Based on the data, Tesla, Inc. (TSLA) appears ..."
1,Apple Inc.,"Based on the provided data, Apple Inc. (AAPL) ..."
2,Alphabet Inc.,Alphabet Inc. (GOOGL) appears to be overvalued...
3,Microsoft Corporation,Microsoft Corporation (MSFT) appears to be ove...
4,NVIDIA Corporation,NVIDIA Corporation (NVDA) appears to be overva...
5,"Amazon.com, Inc.","Based on the data, Amazon.com, Inc. (AMZN) app..."
6,"Meta Platforms, Inc.","Based on the provided data, Meta Platforms, In..."
7,"Netflix, Inc.","Based on the data, Netflix, Inc. (NFLX) appear..."


In [70]:
test_df.to_csv("stock_fundamentals_with_insights.csv", index=False)
test_df.to_excel("stock_fundamentals_with_insights.xlsx", index=False)
